In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as mpl_anim
from IPython.display import HTML
from qutip import *

# 1. Setup Simulation
N = 25              # Hilbert space size
alpha = 2.0         # Coherent State amplitude (Mean photons = 4)
g = 0.05
w = 1.0

# Initial State: Coherent State in Cavity, Atom Ground
psi0 = tensor(coherent(N, alpha), basis(2, 0))

# Hamiltonian (Standard JC)
a  = tensor(destroy(N), qeye(2))
sm = tensor(qeye(N), sigmam())
sz = tensor(qeye(N), sigmaz())
H = w * a.dag() * a + 0.5 * w * sz + g * (a.dag() * sm + a * sm.dag())

# Time Evolution
# Run long enough to see the "smearing" (Collapse)
t_max = 2 * np.pi / g 
frames = 80
tlist = np.linspace(0, t_max, frames)

print("1. Running Cat State Simulation...")
output = mesolve(H, psi0, tlist, [], [])

# Pre-Calculate Wigner Data 
print("2. Calculating Wigner frames")
xvec = np.linspace(-6, 6, 100)
wigner_data = []

for state in output.states:
    rho_cavity = state.ptrace(0)
    wigner_data.append(wigner(rho_cavity, xvec, xvec))

# Build Animation
fig, ax = plt.subplots(figsize=(6, 6))

w_max = np.max(wigner_data[0])
w_min = -w_max

mesh = ax.contourf(xvec, xvec, wigner_data[0], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))
ax.set_aspect('equal')
ax.set_title("Schrödinger Cat Formation")

def update(frame):
    ax.clear()
    ax.contourf(xvec, xvec, wigner_data[frame], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))
    ax.set_title(f"Cat State Dynamics (t={tlist[frame]:.1f})")
    ax.set_xlabel("Position q")
    ax.set_ylabel("Momentum p")
    return ax,

anim = mpl_anim.FuncAnimation(fig, update, frames=frames, interval=80, blit=False)
plt.close()
HTML(anim.to_jshtml())

1. Running Cat State Simulation...
2. Calculating Wigner frames


c:\Users\Vedant\miniconda3\envs\qiskit-ok\lib\site-packages\qutip\solver\solver_base.py:598: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(
